<a href="https://colab.research.google.com/github/hariomshahu/FightDetection/blob/VideoMAE_Transformer/fight_detection_using_VideoMAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install kaggle

In [3]:
!mkdir ~/.kaggle

In [4]:
!cp "/content/drive/MyDrive/Colab Notebooks/kaggle_api_credentials/kaggle.json" ~/.kaggle/kaggle.json

In [5]:
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d vulamnguyen/rwf2000/

Dataset URL: https://www.kaggle.com/datasets/vulamnguyen/rwf2000/versions/
License(s): unknown
100% 11.4G/11.5G [01:14<00:00, 155MB/s]
100% 11.5G/11.5G [01:14<00:00, 165MB/s]


In [7]:
import os

dataset_zip = "rwf2000.zip"

destination_folder = "/content/dataset"

os.makedirs(destination_folder, exist_ok=True)

!unzip -q $dataset_zip -d $destination_folder

error:  cannot create /content/dataset/RWF-2000/train/Fight/1060314╤ДтХХ╨╜╤Е╨│╨▓╤Е╨┤╨╣╤Ж╨й╨п╤Й╨Ц╨╗╤Й╨й╨▓╤Ж╨Р╨╡╤И╨╕тХС╤Е╨╛╨┤╤Ж╨ктФд╤Е╨Ъ╨л╤ДтХС╨Ы╤ДтХЧтХв╤ПтХЭ╨Ш╤Е╨Э╨о╤Й╨Ч╨б╤ЕтФВтЦС╤ПтХЭ╨Щ_17.avi
        File name too long
error:  cannot create /content/dataset/RWF-2000/train/Fight/╤ДтХХ╨┤╤Е╨╡тФВ╤Е╨╜╨а╤ЕтХС╨з╤Й╨Ч╨Ь╤Ж╨Щ╨г╤Ж╨отХв╤ПтХЭ╨Ь╤Е╨ЭтФд╤ЕтХЭ╨е╤Е╨Я╨б╤Ж╨етЦС╤Е╨а╨Э╤З╨дтХЦ╤Е╨╜╨а╤Е╨ЧтХС╤Ж╨Щ╨Ы╤ПтХЭ╨Ь╤Ж╨и╨┐╤ДтХЧ╨Р╤ДтХг╨Ш╤Е╨Ю╨п╤Е╨л╨░╤ПтХЭ╨п╤З╨л╨б╤Ж╨Ю╨╖╤Ж╨Ы╨Э╤ДтХХ╨Ы╤Е╨Х╨╕╤З╨╕╨Ы_urlgot_220.avi
        File name too long
error:  cannot create /content/dataset/RWF-2000/train/Fight/╤ДтХХ╨┤╤Е╨╡тФВ╤Е╨╜╨а╤ЕтХС╨з╤Й╨Ч╨Ь╤Ж╨Щ╨г╤Ж╨отХв╤ПтХЭ╨Ь╤Е╨ЭтФд╤ЕтХЭ╨е╤Е╨Я╨б╤Ж╨етЦС╤Е╨а╨Э╤З╨дтХЦ╤Е╨╜╨а╤Е╨ЧтХС╤Ж╨Щ╨Ы╤ПтХЭ╨Ь╤Ж╨и╨┐╤ДтХЧ╨Р╤ДтХг╨Ш╤Е╨Ю╨п╤Е╨л╨░╤ПтХЭ╨п╤З╨л╨б╤Ж╨Ю╨╖╤Ж╨Ы╨Э╤ДтХХ╨Ы╤Е╨Х╨╕╤З╨╕╨Ы_urlgot_221.avi
        File name too long
error:  cannot create /content/dataset/RWF-2000/train/Fight/╤Е╨а╨Ь╤И╨▒╨Ь╤И╨Р╨С╤Ж╨нтФР╤Е╨Я╨╗╤Ж╨н╨╡╤ЕтХХ╨╛╤ЖтФдтХЫ╤Ж╨Щ╨г╤Ж╨отХв╤ПтХЭ╨Ь╤ДтХХ╨Ы╤ДтХХ╨Р╤З╨╖╨в╤Ж╨м╨Щ╤ДтХСтХС╤И╨╖╨С╤Ж╨╜тХЧ╤З╨╡╨

In [8]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetV2B0
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau


In [25]:
import os
import cv2
import numpy as np
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import VideoMAEImageProcessor, VideoMAEForVideoClassification
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

class VideoDataset(Dataset):
    def __init__(self, video_paths, labels, image_processor, num_frames=16):
        self.video_paths = video_paths
        self.labels = labels
        self.image_processor = image_processor
        self.num_frames = num_frames

    def __len__(self):
        return len(self.video_paths)

    def __getitem__(self, idx):
        video_path = self.video_paths[idx]
        label = self.labels[idx]

        # Load and preprocess video frames
        frames = self._load_video(video_path)

        # Ensure we have exactly num_frames
        if len(frames) < self.num_frames:
            # If we don't have enough frames, duplicate the last frame
            last_frame = frames[-1] if frames else Image.fromarray(np.zeros((224, 224, 3), dtype=np.uint8))
            while len(frames) < self.num_frames:
                frames.append(last_frame)
        elif len(frames) > self.num_frames:
            # If we have too many frames, sample evenly
            indices = np.linspace(0, len(frames)-1, self.num_frames, dtype=int)
            frames = [frames[i] for i in indices]

        # Process frames
        processed = self.image_processor(frames, return_tensors="pt")

        return {
            'pixel_values': processed['pixel_values'][0],
            'label': torch.tensor(label, dtype=torch.long)
        }

    def _load_video(self, video_path):
        cap = cv2.VideoCapture(video_path)
        frames = []

        try:
            while True:
                ret, frame = cap.read()
                if not ret:
                    break

                # Convert to RGB
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                # Resize to 224x224 (VideoMAE default size)
                frame = cv2.resize(frame, (224, 224))

                # Convert to PIL Image
                frame = Image.fromarray(frame)
                frames.append(frame)

        except Exception as e:
            print(f"Error loading video {video_path}: {str(e)}")

        finally:
            cap.release()

        # If no frames were read, create a dummy frame
        if not frames:
            dummy_frame = Image.fromarray(np.zeros((224, 224, 3), dtype=np.uint8))
            frames = [dummy_frame] * self.num_frames

        return frames

def prepare_data(data_dir):
    # Get paths for train set
    train_dir = os.path.join(data_dir, 'train')
    train_fight_dir = os.path.join(train_dir, 'Fight')
    train_nonfight_dir = os.path.join(train_dir, 'NonFight')

    train_fight_videos = [os.path.join(train_fight_dir, f) for f in os.listdir(train_fight_dir) if f.endswith('.avi')]
    train_nonfight_videos = [os.path.join(train_nonfight_dir, f) for f in os.listdir(train_nonfight_dir) if f.endswith('.avi')]

    train_paths = train_fight_videos + train_nonfight_videos
    train_labels = [1] * len(train_fight_videos) + [0] * len(train_nonfight_videos)

    # Get paths for validation set
    val_dir = os.path.join(data_dir, 'val')
    val_fight_dir = os.path.join(val_dir, 'Fight')
    val_nonfight_dir = os.path.join(val_dir, 'NonFight')

    val_fight_videos = [os.path.join(val_fight_dir, f) for f in os.listdir(val_fight_dir) if f.endswith('.avi')]
    val_nonfight_videos = [os.path.join(val_nonfight_dir, f) for f in os.listdir(val_nonfight_dir) if f.endswith('.avi')]

    val_paths = val_fight_videos + val_nonfight_videos
    val_labels = [1] * len(val_fight_videos) + [0] * len(val_nonfight_videos)

    return train_paths, train_labels, val_paths, val_labels

def train_model(train_dataset, val_dataset, num_epochs=10, batch_size=2):  # Reduced batch size
    # Create data loaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=2,
        pin_memory=True if torch.cuda.is_available() else False
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        num_workers=2,
        pin_memory=True if torch.cuda.is_available() else False
    )

    # Initialize model
    model = VideoMAEForVideoClassification.from_pretrained(
        "MCG-NJU/videomae-base-finetuned-kinetics",
        num_labels=2,
        label2id={"NonFight": 0, "Fight": 1},
        id2label={0: "NonFight", 1: "Fight"},
        ignore_mismatched_sizes=True
    )

    # Move model to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    # Initialize optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

    # Training loop
    best_val_accuracy = 0

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")

        # Training phase
        model.train()
        train_loss = 0
        for batch in train_loader:
            pixel_values = batch['pixel_values'].to(device)
            labels = batch['label'].to(device)

            outputs = model(pixel_values=pixel_values, labels=labels)
            loss = outputs.loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)
        print(f"Average training loss: {avg_train_loss:.4f}")

        # Validation phase
        model.eval()
        val_preds = []
        val_labels = []

        with torch.no_grad():
            for batch in val_loader:
                pixel_values = batch['pixel_values'].to(device)
                labels = batch['label'].to(device)

                outputs = model(pixel_values=pixel_values)
                predictions = torch.argmax(outputs.logits, dim=-1)

                val_preds.extend(predictions.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())

        # Calculate metrics
        val_accuracy = accuracy_score(val_labels, val_preds)
        precision, recall, f1, _ = precision_recall_fscore_support(val_labels, val_preds, average='binary')

        print(f"Validation Metrics:")
        print(f"Accuracy: {val_accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1-score: {f1:.4f}")

        # Save best model
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            model.save_pretrained("best_model")
            print(f"New best model saved with validation accuracy: {val_accuracy:.4f}")

    return model

def main():
    # Set random seeds
    torch.manual_seed(42)
    np.random.seed(42)

    # Print GPU information
    print("GPU Available:", torch.cuda.is_available())
    if torch.cuda.is_available():
        print("GPU:", torch.cuda.get_device_name(0))
        print("Memory:", torch.cuda.get_device_properties(0).total_memory / 1e9, "GB")

    # Initialize the image processor
    image_processor = VideoMAEImageProcessor.from_pretrained("MCG-NJU/videomae-base-finetuned-kinetics")
    print("\nImage Processor Config:", image_processor.image_mean, image_processor.image_std)

    # Prepare data
    print("\nPreparing datasets...")
    data_dir = '/content/dataset/RWF-2000'
    train_paths, train_labels, val_paths, val_labels = prepare_data(data_dir)
    print(f"Found {len(train_paths)} training videos and {len(val_paths)} validation videos")

    # Create datasets
    train_dataset = VideoDataset(train_paths, train_labels, image_processor)
    val_dataset = VideoDataset(val_paths, val_labels, image_processor)
    print("Datasets created successfully")

    # Train model
    print("\nStarting training...")
    model = train_model(train_dataset, val_dataset)
    print("Training completed!")

if __name__ == "__main__":
    main()

GPU Available: True
GPU: NVIDIA A100-SXM4-40GB
Memory: 42.481811456 GB

Image Processor Config: [0.485, 0.456, 0.406] [0.229, 0.224, 0.225]

Preparing datasets...
Found 1576 training videos and 400 validation videos
Datasets created successfully

Starting training...


Some weights of VideoMAEForVideoClassification were not initialized from the model checkpoint at MCG-NJU/videomae-base-finetuned-kinetics and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([400]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([400, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Epoch 1/10
Average training loss: 0.6127
Validation Metrics:
Accuracy: 0.7600
Precision: 0.7342
Recall: 0.8150
F1-score: 0.7725
New best model saved with validation accuracy: 0.7600

Epoch 2/10
Average training loss: 0.5601
Validation Metrics:
Accuracy: 0.7300
Precision: 0.6597
Recall: 0.9500
F1-score: 0.7787

Epoch 3/10
Average training loss: 0.5126
Validation Metrics:
Accuracy: 0.7375
Precision: 0.8276
Recall: 0.6000
F1-score: 0.6957

Epoch 4/10
Average training loss: 0.4496
Validation Metrics:
Accuracy: 0.7200
Precision: 0.8333
Recall: 0.5500
F1-score: 0.6627

Epoch 5/10
Average training loss: 0.4003
Validation Metrics:
Accuracy: 0.6825
Precision: 0.8349
Recall: 0.4550
F1-score: 0.5890

Epoch 6/10
Average training loss: 0.3457
Validation Metrics:
Accuracy: 0.7725
Precision: 0.7559
Recall: 0.8050
F1-score: 0.7797
New best model saved with validation accuracy: 0.7725

Epoch 7/10
Average training loss: 0.2941
Validation Metrics:
Accuracy: 0.7050
Precision: 0.8475
Recall: 0.5000
F1-sco